<a href="https://colab.research.google.com/github/Deepak-Mewada/Advance-Machine-Learning/blob/main/Download_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive

drive.mount('/content/gdrive/', force_remount=True)

MessageError: Error: credential propagation was unsuccessful

In [ ]:
%cd gdrive/MyDrive/EEGdatasets

In [ ]:
! pip install moabb

In [ ]:
import moabb
from moabb.datasets import BNCI2014001
from moabb.paradigms import MotorImagery
import numpy as np
import os

In [ ]:
def load_data(root, data_path, dataset, prgm_MI_classes, subj, labels_unique, num_trials):
    if not os.path.exists(os.path.join(root, data_path)):
        os.mkdir(os.path.join(root, data_path))
    temp = dict()
    Xs, labels, m_ = prgm_MI_classes.get_data(dataset=dataset, subjects=[subj])
    if labels_unique is None:
        labels_unique = np.unique(labels)
    for (data, label) in zip(Xs, labels):
        if label in temp:
            temp[label].append(data)
        else:
            temp[label] = [data]
    for i, label in enumerate(labels_unique):
        x = np.array(temp[label][:num_trials])
        for j in range(len(x)):
            np.save(os.path.join(root, data_path, f"s{subj}_c{i}_t{j}.npy"), x[j])
    del temp
    del Xs
    del labels
    del m_
    return labels_unique

In [ ]:
def load_bciiv2a(train_subj, test_subj):
    root = ''
    data_npy = 'bciiv2a'
    dataset = BNCI2014001()

    fmin, fmax = 4, 32
    raw = dataset.get_data(subjects=[1])[1]['session_T']['run_1']
    dataset_channels = raw.pick_types(eeg=True).ch_names
    sfreq = 250.
    prgm_MI_classes = MotorImagery(n_classes=4, channels=dataset_channels, resample=sfreq, fmin=fmin, fmax=fmax)

    labels_unique = None
    for subj in train_subj:
        labels_unique = load_data(root, data_npy+'_train', dataset, prgm_MI_classes, subj, labels_unique, 144)
    for subj in test_subj:
        labels_unique = load_data(root, data_npy+'_test', dataset, prgm_MI_classes, subj, labels_unique, 144)

In [ ]:
load_bciiv2a(train_subj=list(range(1, 10)), test_subj=[])